<a href="https://colab.research.google.com/github/Ajay-user/DataScience/blob/master/Quick_start_for_beginners_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Build a neural network that classifies images.

> Train this neural network.

> And, finally, evaluate the accuracy of the model.

In [1]:
import tensorflow as tf

Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers:

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
X_train, X_test = X_train/255.0 , X_test/255.0

Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:

In [6]:
input_shape = X_train.shape[1:]

n_classes = 10

input_shape

(28, 28)

In [13]:
model  = tf.keras.Sequential([
                              tf.keras.layers.Flatten(input_shape=input_shape),
                              tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
                              tf.keras.layers.Dropout(.2),
                              tf.keras.layers.Dense(n_classes)

])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [19]:
predictions = model(X_train[:1]).numpy()
predictions

array([[-0.7408574 , -0.26374465, -0.5726848 , -0.09695703,  0.03974769,
        -0.14470729, -0.4762886 ,  0.13290104,  0.14810881,  1.0120139 ]],
      dtype=float32)

The tf.nn.softmax function converts these logits to "probabilities" for each class:

In [22]:
tf.nn.softmax(predictions).numpy()

array([[0.04629868, 0.07460639, 0.05477785, 0.0881477 , 0.10106041,
        0.08403753, 0.06032111, 0.11092695, 0.11262679, 0.2671966 ]],
      dtype=float32)

Note: It is possible to bake this tf.nn.softmax in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, `this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.`

The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [27]:
# This loss is equal to the negative log probability of the true class
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# the untrained model gives probabilities close to random
-tf.math.log(1/10).numpy()

2.3025851

In [29]:
loss_fn(y_train[:1], predictions).numpy()

2.476492

In [30]:
model.compile(optimizer='Adam', loss=loss_fn, metrics=['accuracy'])

The Model.fit method adjusts the model parameters to minimize the loss:

In [31]:
model.fit(X_train, y_train , epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4948 - accuracy: 0.8539
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1530 - accuracy: 0.9549
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1089 - accuracy: 0.9669
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0840 - accuracy: 0.9742
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0735 - accuracy: 0.9768


The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".

In [32]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.0756 - accuracy: 0.9766


[0.07561255246400833, 0.9765999913215637]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [33]:
probability_model = tf.keras.Sequential([
                                         model,
                                         tf.keras.layers.Softmax()
])

In [41]:
y_preds = probability_model.predict(X_test)